In [16]:
import os
from dotenv import load_dotenv

# Set environment variables in the .env file.
load_dotenv()

CHAT_MODEL = os.environ["OPENAI_CHAT_MODEL"]
CHAT_DEPLOYMENT = os.environ["OPENAI_CHAT_DEPLOYMENT"]
OPENAI_API_VERSION = os.environ["OPENAI_API_VERSION"]

SQL_DB_USER = os.environ["SQL_DB_USER"]
SQL_DB_PASSWORD = os.environ["SQL_DB_PASSWORD"]
SQL_DB_SERVER_NAME = os.environ["SQL_DB_SERVER_NAME"]
SQL_DB_NAME = os.environ["SQL_DB_NAME"]
SQL_CONNECTIONSTRING_FORMAT = os.environ["SQL_CONNECTIONSTRING_FORMAT"]

In [17]:
from langchain.chat_models import AzureChatOpenAI

chat = AzureChatOpenAI(
    deployment_name=CHAT_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION,
    #temperature=0.3,
    verbose=True
)

In [18]:
from langchain.sql_database import SQLDatabase

connection_string = SQL_CONNECTIONSTRING_FORMAT.format(
    database_user=SQL_DB_USER,
    database_password=SQL_DB_PASSWORD,
    database_server=SQL_DB_SERVER_NAME,
    database_db=SQL_DB_NAME)

db = SQLDatabase.from_uri(connection_string)

In [19]:
sql = "exec [dbo].[getCustomerId] @FirstName='Maria', @LastName='Anders'"
db.run(sql)

'[(1,)]'

In [20]:
results = db._execute(sql)
results[0][0]

1

In [21]:
def execute_storedprocedure(db, stored_procedure_name, *args, schema='dbo'):
    sql_string = "exec [{schema}].[{stored_procedure_name}] ".format(schema=schema, stored_procedure_name=stored_procedure_name)
    for i in range(0, len(args), 2):
        param_name = args[i]
        param_value = args[i+1]
        sql_string += "@{param_name}='{param_value}', ".format(param_name=param_name, param_value=param_value)
    sql_string = sql_string[:-2]
    
    return db._execute(sql_string)

In [24]:
resultsTupple = execute_storedprocedure(db, 'getCustomerId', 'FirstName', 'Maria', 'LastName', 'Anders')

print(resultsTupple[0][0])
print(str(resultsTupple))

1
[(1,)]
